In [2]:
print("Hello World")

Hello World


In [3]:
%pip install -qU langchain "langchain[anthropic]"

In [3]:
%pip install langchain langchain-groq

## Building Basic Agents


In [5]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in Bengali.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""


In [6]:
from dataclasses import dataclass
from langchain.tools import tool,ToolRuntime


In [7]:
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

In [8]:
from langchain_groq import ChatGroq

In [13]:
model = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0,
    max_retries=2,
)

In [25]:
result=model.invoke("Hello, how are you? Answer me in bengali")
print(result)

content='আমি ভালো আছি, ধন্যবাদ।' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 47, 'total_tokens': 75, 'completion_time': 0.035262896, 'completion_tokens_details': None, 'prompt_time': 0.0022946, 'prompt_tokens_details': None, 'queue_time': 0.05651787, 'total_time': 0.037557496}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--87a4a114-bf74-4c7f-ab73-42badc05a197-0' usage_metadata={'input_tokens': 47, 'output_tokens': 28, 'total_tokens': 75}


In [23]:
print(list(result)[0][1])

আমি ভালো আছি, ধন্যবাদ।


In [26]:
from dataclasses import dataclass

# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

In [27]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

## Building The Main Agent


In [28]:
from langchain.agents.structured_output import ToolStrategy

In [29]:
from langchain.agents import create_agent

In [31]:
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_weather_for_location,get_user_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
)

In [32]:
config = {"configurable":{"thred_id":"1"}}

In [ ]:
responses=agent.invoke(
    {
        "messages":[
            {
                "role": "user",
                "content": "What is the weather like in New York?"
            }
        ]
    },
    config=config,
    context=Context(user_id="1")

)
print(responses)

In [38]:
from dataclasses import dataclass

from langchain_groq import ChatGroq
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy


# -------------------------
# SYSTEM PROMPT
# -------------------------

SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in english.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location.
"""


# -------------------------
# CONTEXT SCHEMA
# -------------------------

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str


# -------------------------
# TOOL DEFINITIONS
# -------------------------

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"


# -------------------------
# GROQ MODEL
# -------------------------

model = ChatGroq(
    model="llama-3.1-8b-instant",   # You can change to any Groq LLM
    temperature=0
)


# -------------------------
# STRUCTURED OUTPUT FORMAT
# -------------------------

@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    punny_response: str
    weather_conditions: str | None = None



# -------------------------
# MEMORY CHECKPOINT
# -------------------------

checkpointer = InMemorySaver()


# -------------------------
# CREATE AGENT
# -------------------------

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)


# -------------------------
# RUN AGENT - FIRST REQUEST
# -------------------------

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response["structured_response"])






ResponseFormat(punny_response="I'm not sure, but I can try to forecast a response for you!", weather_conditions=None)


In [39]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Florida"}]},
    config=config,
    context=Context(user_id="1")
)

print(response["structured_response"])

ResponseFormat(punny_response="Looks like you're in the Sunshine State! I'm forecasting partly cloudy skies with a high of 78 and a low of 62.", weather_conditions='partly cloudy skies with a high of 78 and a low of 62.')


In [41]:
tools=[get_user_location,get_weather_for_location]

In [5]:
from langchain_groq import ChatGroq

Structured output
Models can be requested to provide their response in a format matching a given schema. This is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.


In [6]:
model = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [8]:
from pydantic import BaseModel,Field


In [14]:
class Movie(BaseModel):
    """Movie model"""
    title: str = Field(..., description="Title of the movie")
    year: int = Field(..., description="Year of the movie")

In [15]:
model_with_structure=model.with_structured_output(Movie)

In [ ]:
response=model_with_structure.invoke(
    {"messages": [{"role": "user", "content": "Provide details about the movie Inception"}]}
)

print(response)

title='Inception' year=2010


In [13]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

title='Inception' year=2010 director='Christopher Nolan' rating=8.8
